In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
from sklearn.model_selection import GroupKFold
from catboost import CatBoostRegressor, Pool

def nmae(true, pred):
    score = np.mean((np.abs(true-pred))/true)
    return score
    
df_train = pd.read_csv('/content/train_ff.csv')
df_test = pd.read_csv('/content/test_ff.csv')
df_train
categorical = ['일출 시', '구분', '공휴일', 'month', 'day', '시간', 'weekday']

In [ ]:
group1 = [0,1,3,4,5]
group2 = [2,6]
df_train['연월일'] = pd.to_datetime(df_train['연월일'])
df_test['연월일'] = pd.to_datetime(df_test['연월일'])

groups1 = ((df_train[df_train['구분'].isin(group1)]['연월일'].dt.year-2013)*12 + df_train[df_train['구분'].isin(group1)]['연월일'].dt.month).reset_index(drop=True)
y_labels1 = df_train[df_train['구분'].isin(group1)]['공급량']
X_train1 = df_train[df_train['구분'].isin(group1)].drop(columns=['연월일','공급량']).reset_index(drop=True)
X_test1 = df_test[df_test['구분'].isin(group1)].drop(columns=['연월일'])
groups2 = ((df_train[df_train['구분'].isin(group2)]['연월일'].dt.year-2013)*12 + df_train[df_train['구분'].isin(group2)]['연월일'].dt.month).reset_index(drop=True)
y_labels2 = df_train[df_train['구분'].isin(group2)]['공급량']
X_train2 = df_train[df_train['구분'].isin(group2)].drop(columns=['연월일','공급량']).reset_index(drop=True)
X_test2 = df_test[df_test['구분'].isin(group2)].drop(columns=['연월일'])

In [ ]:
kf = GroupKFold(5)
cb = CatBoostRegressor(task_type="GPU", iterations = 10000, learning_rate = 0.03, depth = 3, eval_metric = 'MAE', loss_function = 'MAE', random_seed = 42)

cb_pred_1 = []
mae_list = []
for tr_idx, val_idx in kf.split(X_train1,y_labels1,groups1 ):
    tr_x, val_x = X_train1.iloc[tr_idx], X_train1.iloc[val_idx]
    tr_y, val_y = y_labels1.iloc[tr_idx], y_labels1.iloc[val_idx]
    cb.fit(tr_x, tr_y, eval_set=(val_x,val_y), cat_features=categorical, early_stopping_rounds = 1000, use_best_model=True, verbose=2000)
    # train_data = Pool(data = tr_x, label = tr_y, cat_features = categorical)
    # val_data = Pool(data = val_x, label = val_y, cat_features = categorical)
    # cb.fit(train_data, eval_set = val_data, early_stopping_rounds = 2000, use_best_model = True, verbose = 1000)
    best = cb.best_iteration_
    pred = cb.predict(val_x, ntree_end = best)
    mae = nmae(val_y, pred)
    mae_list.append(mae)
    print(f'FOLD NMAE = {mae}')
    sub_pred = cb.predict(X_test1, ntree_end = best)
    cb_pred_1.append(sub_pred)
print(f'NMAE = {np.mean(np.array(mae_list))}')

In [ ]:
kf = GroupKFold(5)
cb = CatBoostRegressor(task_type="GPU", iterations = 10000, learning_rate = 0.03, depth = 3, eval_metric = 'MAE', loss_function = 'MAE')

cb_pred_2 = []
mae_list2 = []
for tr_idx, val_idx in kf.split(X_train2,y_labels2,groups2):
    tr_x, val_x = X_train2.iloc[tr_idx], X_train2.iloc[val_idx]
    tr_y, val_y = y_labels2.iloc[tr_idx], y_labels2.iloc[val_idx]
    # train_data = Pool(data = tr_x, label = tr_y)
    # val_data = Pool(data = val_x, label = val_y)
    cb.fit(tr_x, tr_y, eval_set=(val_x,val_y), cat_features=categorical, early_stopping_rounds = 1000, use_best_model=True, verbose=2000)
    best = cb.best_iteration_
    pred = cb.predict(val_x, ntree_end = best)
    mae = nmae(val_y, pred)
    mae_list2.append(mae)
    print(f'FOLD NMAE = {mae}')
    sub_pred = cb.predict(X_test2, ntree_end = best)
    cb_pred_2.append(sub_pred)
print(f'NMAE = {np.mean(np.array(mae_list2))}')

In [ ]:
X_test2['공급량'] = (cb_pred_2[0] + cb_pred_2[1] + cb_pred_2[2] + cb_pred_2[3] + cb_pred_2[4])/5
X_test1['공급량'] = (cb_pred_1[0] + cb_pred_1[1] + cb_pred_1[2] + cb_pred_1[3] + cb_pred_1[4])/5
X_test1['연월일'] = df_test[df_test['구분'].isin(group1)]['연월일']
X_test2['연월일'] = df_test[df_test['구분'].isin(group2)]['연월일']

In [ ]:
result = pd.concat([X_test1, X_test2])
result = result[['연월일', '시간', '구분', '공급량']]
result

In [ ]:
PATH = "/content/drive/MyDrive/dacon/gas/data"
submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
submission['연월일'] = submission['일자|시간|구분'].str.split(' ').str[0]
submission['시간'] = submission['일자|시간|구분'].str.split(' ').str[1].astype(int)
submission['구분'] = submission['일자|시간|구분'].str.split(' ').str[2]
submission = submission.drop(columns=['공급량'])
submission['연월일'] = pd.to_datetime(submission['연월일'])
submission['시간'] = submission['시간'].astype(int)
submission['구분'] = submission['구분'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'G': 5, 'H': 6})
submission = pd.merge(submission, result, how='left', on=['연월일', '시간', '구분'])
pred = submission['공급량']

PATH = "/content/drive/MyDrive/dacon/gas/data"
submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
submission['공급량'] = pred
submission.to_csv(os.path.join("/content/drive/MyDrive/dacon/gas/submission", "2staged_cat_me_byme.csv"), index=False)